In [ ]:
import json
import re
import time
import logging
from logging.handlers import RotatingFileHandler
import os

# ================= CONFIGURAZIONE =================
# IL PERCORSO DEVE ESSERE LO STESSO CHE HAI MESSO NELL'AGENT WAZUH
LOG_FILE_WAZUH = r"C:\Logs\telegram_cti.json"

# Assicuriamoci che la cartella esista
log_dir = os.path.dirname(LOG_FILE_WAZUH)
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

# Configurazione Logger
logger = logging.getLogger("WazuhSimulator")
logger.setLevel(logging.INFO)
handler = RotatingFileHandler(LOG_FILE_WAZUH, maxBytes=10*1024*1024, backupCount=5)
logger.addHandler(handler)

# ================= L'INPUT CHE MI HAI DATO =================
raw_output = """
Checking 1/23... [IP_ADDRESSE] 185.146.173.58 -> MALICIOUS (1/95)
    ℹ️  DETAILS:
       🌍  Owner/ASN: CLOUDFLARENET
       🏳️  Country: SE

Checking 2/23... [FILE] AB4F368345DC20DC59516C098AAD331C -> Not Found
Checking 3/23... [URL] https://happytiger.co.uk -> MALICIOUS (2/98)
    ℹ️  DETAILS:
       🏷  Tags: external-resources
       📑  Page Title: Just a moment...
       🗂  Categories: Gambling (alphaMountain.ai), shopping, gambling

Checking 4/23... [IP_ADDRESSE] 51.158.150.210 -> MALICIOUS (1/95)
    ℹ️  DETAILS:
       🌍  Owner/ASN: Scaleway S.a.s.
       🏳️  Country: NL

Checking 5/23... [IP_ADDRESSE] 94.228.165.234 -> CLEAN
Checking 6/23... [IP_ADDRESSE] 49.13.69.46 -> CLEAN
Checking 11/23... [URL] https://w0rm.pw -> MALICIOUS (1/96)
    ℹ️  DETAILS:
       🏷  Tags: opendir
       📑  Page Title: Index of /

Checking 12/23... [URL] http://213.249.19.49:8080 -> CLEAN
Checking 13/23... [IP_ADDRESSE] 213.249.19.49 -> CLEAN
Checking 14/23... [IP_ADDRESSE] 185.146.173.59 -> MALICIOUS (1/95)
    ℹ️  DETAILS:
       🌍  Owner/ASN: CLOUDFLARENET
       🏳️  Country: SE

Checking 18/23... [IP_ADDRESSE] 160.153.0.179 -> MALICIOUS (1/95)
    ℹ️  DETAILS:
       🌍  Owner/ASN: Cloudflare London, LLC
       🏳️  Country: US
"""

def parse_and_feed():
    print(f"🚀 Avvio simulazione feed verso: {LOG_FILE_WAZUH}")
    
    # Dividiamo l'input per "Checking" per isolare i blocchi
    blocks = raw_output.strip().split("Checking")
    
    events_sent = 0
    
    for block in blocks:
        if not block.strip(): continue
        
        # Pulizia testo
        block = "Checking" + block # Rimettiamo la parola chiave
        lines = block.strip().split('\n')
        header = lines[0]
        
        # Cerca solo quelli MALICIOUS
        if "MALICIOUS" in header:
            # Estrazione Dati con Regex
            # Esempio header: Checking 1/23... [IP_ADDRESSE] 185.146.173.58 -> MALICIOUS (1/95)
            match = re.search(r"\[(.*?)\]\s+(.*?)\s+->\s+MALICIOUS\s+\((.*?)\)", header)
            
            if match:
                ioc_type = match.group(1)
                ioc_value = match.group(2)
                vt_score = match.group(3)
                
                # Cerchiamo dettagli extra nel resto del blocco (Paese, ASN)
                country = "Unknown"
                owner = "Unknown"
                for line in lines:
                    if "Country:" in line:
                        country = line.split("Country:")[1].strip()
                    if "Owner/ASN:" in line:
                        owner = line.split("Owner/ASN:")[1].strip()

# --- CREAZIONE EVENTO JSON PER WAZUH ---
                threat_event = {
                    "timestamp": time.strftime("%Y-%m-%dT%H:%M:%SZ", time.gmtime()),
                    "application": "Telegram_CTI_Bot",  # Deve matchare la regola Wazuh!
                    "severity": "Critical",
                    "rule_name": "Manual_Test_Injection",
                    
                    # Dati Finti per simulare il contesto del gruppo Telegram
                    "group_name": "TEST_MANUALE_WAZUH", 
                    "topic": "Security_Test",
                    "country": country,
                    "user_id": 999999,
                    "cyber_score": 0.99, # Mettiamo alto per far scattare l'allarme
                    
                    # Dati Veri estratti dall'output
                    "ioc_type": ioc_type,
                    "ioc_value": ioc_value,
                    "virustotal_score": vt_score,
                    "owner_asn": owner,
                    
                    "message_text": f"Simulazione rilevamento IoC: {ioc_value}",
                    "context_prev": "Test manuale | Nessun contesto reale"
                }
                
                # Scrittura su File
                logger.info(json.dumps(threat_event))
                print(f"   🚨 Inviato allarme per: {ioc_value} ({vt_score})")
                events_sent += 1
                
                # Aspettiamo 2 secondi tra uno e l'altro per vederli apparire in ordine
                time.sleep(2)

    print(f"\n✅ Finito! Inviati {events_sent} eventi a Wazuh.")


parse_and_feed()